
<h1 id="Tutorial-6---Three-way-branch-migration-trajectories">Tutorial 6 - Three-way branch migration trajectories<a class="anchor-link" href="#Tutorial-6---Three-way-branch-migration-trajectories">¶</a></h1>



<p>This example illustrates how to examine trajectories in multistranded simulations, where association and dissociation can change the number and type of complexes.  Pay attention to how, when that happens, the strand ordering can change, and thus dot-paren structures must be displayed differently.</p>


In [1]:
from multistrand.objects import *
from multistrand.options import Options, EnergyType
from multistrand.system import SimSystem, calculate_energy


<p>Setup options for the simulation.</p>


In [2]:
def create_setup():
    # build complexes with domain-level information
    toehold_seq = "GTGGGT"
    bm_design_A = "ACCGCACGTCCACGGTGTCG"
    bm_design_B = "ACCGCACGTCACTCACCTCG"

    toehold = Domain(name="toehold",sequence=toehold_seq)
    branch_migration_A = Domain(name="bm_A", sequence=bm_design_A)
    branch_migration_B = Domain(name="bm_B", sequence=bm_design_B)
    
    substrate_A = toehold + branch_migration_A
    substrate_B = toehold + branch_migration_B
    incumbent_A = Strand(name="incumbent",domains=[branch_migration_A.C])
    incumbent_B = Strand(name="incumbent",domains=[branch_migration_B.C])

    incoming_A = substrate_A.C
    incoming_B = substrate_B.C

    # Note that "+" is used to indicate strand breaks.
    # So the initial structures represent the incoming strand bound by its toehold,
    # and we'll see that either it completes strand displacement, or it dissociates.
    start_complex_A = Complex(strands=[incoming_A, substrate_A, incumbent_A],
                              structure=".(+)(+)")
    start_complex_B = Complex(strands=[incoming_B, substrate_B, incumbent_B],
                              structure=".(+)(+)")
    
    o1 = Options()
    o1.simulation_mode = "Trajectory"
    o1.num_simulations = 1
    o1.simulation_time = 5e-5
    o1.temperature = 37.0
    o1.dangles = 1
    o1.output_interval = 1000   # record every 1000 steps (so we'll get around 100 record entries)
    o1.start_state = [start_complex_A]
    o1.join_concentration=1e-6  # 1 uM
    o1.JSMetropolis37()
    o1.verbosity = 0

    o2 = Options()
    o2.simulation_mode = "Trajectory"    
    o2.num_simulations = 1
    o2.simulation_time = 5e-5
    o2.temperature = 37.0
    o2.dangles = 1
    o2.start_state = [start_complex_B]
    o2.output_interval = 1000   # could do o2.output_time to get trajectory items evenly spaced in time instead of by number of steps
    o2.join_concentration=1e-6  # 1 uM
    o2.JSMetropolis37()
    o2.verbosity = 0
    
    return o1, o2

In [3]:
# generalized from "hairpin trajectories tutorial" version to allow multistrand complexes and multiple complexes in a tube
def print_trajectory(o):
    seqstring=''
    for i in range(len(o.full_trajectory)): # go through each output microstate of the trajectory
        time = o.full_trajectory_times[i]   # time at which this microstate is entered
        states = o.full_trajectory[i]       # this is a list of the complexes present in this tube microstate
        newseqs = []
        for state in states: newseqs += [ state[3] ]   # extract the strand sequences in each complex (joined by "+" for multistranded complexes)
        newseqstring = ' '.join(newseqs)    # make a space-separated string of complexes, to represent the whole tube system sequence
        if not newseqstring == seqstring : 
            print(newseqstring)
            seqstring=newseqstring          # because strand order can change upon association of dissociation, print it when it changes
        structs = []
        for state in states: structs += [ state[4] ]   # similarly extract the secondary structures for each complex
        tubestruct = ' '.join(structs)      # give the dot-paren secondary structure for the whole test tube
        dG=0
        for state in states: dG += state[5]
        print('%s t=%11.9f seconds, dG=%6.2f kcal/mol' % (tubestruct,time, dG))

        # Needlessly verify that the reported trajectory energies are the Tube_Energy values
        dGv=0
        for state in states:
            cs=state[3].split('+')
            st=state[4]
            dGv += calculate_energy( [Complex( strands=[Strand(sequence=s) for s in cs], structure=st)], o, EnergyType.tube)[0]  
        if not dGv == dG: print("Energy Mismatch")


<p>Perform the simulations.</p>


In [4]:
o1,o2 = create_setup()
s1 = SimSystem(o1)
s1.start()
s2 = SimSystem(o2)
s2.start()


<p>Show what happened.</p>


In [5]:
print()
print("Sequence Design 1 (shown every 1000 steps):")
print_trajectory(o1)
print()
print("Sequence Design 2 (shown every 1000 steps):")
print_trajectory(o2)


Sequence Design 1 (shown every 1000 steps):
CGACACCGTGGACGTGCGGTACCCAC+GTGGGTACCGCACGTCCACGGTGTCG+CGACACCGTGGACGTGCGGT
....................((((((+))))))((((((((((((((((((((+)))))))))))))))))))) t=0.000000000 seconds, dG=-19.70 kcal/mol
....((((........))))((((((+))))))((((((((((((((((((((+)))))))))))))))))))) t=0.000000342 seconds, dG=-21.41 kcal/mol
....((((((....)))))).(((((+)))))..(((((((((((((((((((+))))))))))))))))))). t=0.000000691 seconds, dG=-21.72 kcal/mol
....(((((......)))))((((((+)))))).(((((((((((((((((((+))))))))))))))))))). t=0.000001058 seconds, dG=-21.82 kcal/mol
....((((((....))))))((((((+)))))).((((((((((((((((((.+.)))))))))))))))))). t=0.000001402 seconds, dG=-19.98 kcal/mol
.....(((((....))))).((((((+)))))).(((((((((((((((((((+))))))))))))))))))). t=0.000001776 seconds, dG=-21.66 kcal/mol
.....(((((....)))))..(((..+..))).((((((((((((((((((((+)))))))))))))))))))) t=0.000002108 seconds, dG=-20.20 kcal/mol
....(((((......))))).(((..+..)))..((((((((((((((((((.+.))))))


<h3 id="Discussion">Discussion<a class="anchor-link" href="#Discussion">¶</a></h3>



<p>Can you tell the difference between design 1 and design 2?  On most runs, Design 2 finishes within the allotted time.  That is, the incumbent strand gets displaced.  You can see that a "+" turns into a " ", indicating that separate complexes are present.  Or did the incoming strand get rejected, the toehold spontaneously dissociating?  Can you tell?  And can you discern why Design 1 almost never completes displacement in the given time?</p>



<h3 id="Some-notes-for-the-intrepid-explorer:">Some notes for the intrepid explorer:<a class="anchor-link" href="#Some-notes-for-the-intrepid-explorer:">¶</a></h3>


<p>Once <code>s.start()</code> has run, you cannot run <code>start()</code> for this <code>SimSystem</code> again.  You must make a new <code>Options</code> object, or otherwise retrieve the simulation results before you reuse the old <code>Options</code> object, and then create a new <code>SimSystem</code> object.
<p>Note that if <code>num_simulations &gt; 1</code>, then all the recorded states get collected together into the <code>full_trajectory</code>.  To tell where one trajectory stops and the next simulation starts, you can look at the time stamps.</p>
<p>A good exercise to the reader, as a test of understanding: You should be able to extract sequences &amp; structures from trajectory, make a <code>Complex</code> of them, evaluate the energy, start new simulation, etc.</p>
<p>Trajectory like this were used in Schaeffer's MS thesis.  Note that in the thesis, we displayed them in 5'-&gt;3' counterclockwise. Oops.</p>